# Convolutional Neural Network for CIFAR-10
*MC25905 LI Lingxuan*

## 1.测试GPU是否可以使用
使用的CIFAR-10数据集，该数据集的图像大小为32x32x3。在训练过程中最好使用GPU来加快训练过程，而非CPU

In [1]:
import torch
import numpy as np

# 是否可以使用GPU进行训练
train_gpu = torch.cuda.is_available()
if not train_gpu:
    print('CUDA is not available.')
else:
    print('CUDA is available!')

CUDA is not available.


使用设备是笔记本电脑YOGA14s锐龙标压核显版（旧款）：处理器为R7-4800H，无独立显卡，为Vega核显。没有独立显卡，无法使用GPU。

## 2.下载CIFAR-10数据集
该数据集是一个用于识别普适物体的小型数据集。一共包含10个类别的RGB彩色图片，飞机（ airplane ）、汽车（ automobile ）、鸟类（ bird ）、猫（ cat ）、鹿（ deer ）、狗（ dog ）、蛙类（ frog ）、马（ horse ）、船（ ship ）和卡车（ truck ）。图片的尺寸为 32×32 ，数据集中一共有 50000 张训练图片和 10000 张测试图片。
下图是列举了10各类，每一类展示随机的10张图片。


![](https://raw.githubusercontent.com/lilingxuan-dog/7009-neutral-network/main/project2/%E5%9B%BE%E7%89%871.png)

In [4]:
# 加载训练和测试数据，将训练数据分为训练集和验证集
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

#加载数据
num_workers = 0
#每批加载16张图片
batch_size = 16
# percentage of training set to use as validation
valid_size = 0.2

#将数据转换为torch.FloatTensor,并标准化
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

#选择对应训练集与测试集的数据
train_data = datasets.CIFAR10(
    'data',train=True,
    download=True,transform=transform
)
test_data = datasets.CIFAR10(
    'data',train=False,download=True,transform=transform
)

#获取将用于验证的训练指标
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int (np.floor(valid_size*num_train))
train_idx,valid_idx = indices[split:],indices[:split]

#define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

#对数据进行装载，利用batch_size来确认每个包的大小
train_loader = torch.utils.data.DataLoader(train_data,batch_size=batch_size,
                                           sampler=train_sampler,num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data,batch_size=batch_size,
                                           sampler=valid_sampler,num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data,batch_size=batch_size,
                                          num_workers=num_workers)

#图像分类为10个类别
img_class = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']


examples = iter(test_loader)
images, labels = examples.next()


Files already downloaded and verified
Files already downloaded and verified


## 3.搭建卷积神经网络
在pytorch中使用nn.Conv2d定义卷积层，并指定以下参数：
nn.Conv2d(in_channels, out_channels, kernel_size, stride=1,padding=1)

- in_channels 是指输入深度。输入图像中的通道数，对于灰阶图像来说，深度 = 1
- out_channels 是指输出深度，卷积产生的通道数，或你希望获得的过滤图像数量
- kernel_size 是卷积核的大小（通常为 3，表示 3x3 核）
- stride 和 padding 具有默认值：stride=1，padding=0.但是应该根据你希望输出在空间维度 x, y 里具有的大小设置它们的值。



### 为提高测试精度，改进模型从而缓解过拟合：
- **加入Dropout** 让一部分的神经元不参加训练，随即丢掉一些神经元，从而防止过拟合，提高模型的泛化能力，

In [6]:
import torch.nn as nn
import torch.nn.functional as F

# 搭建卷积神经网络
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        # 卷积层&池化层
        #32x32x3
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 3, stride=1,padding=1)

        #16x16x64
        self.conv2 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride=1,padding=1)

        #8x8x128
        self.conv3 = nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, stride=1,padding=1)
        #池化层
        self.pool = nn.MaxPool2d(2,2)
        #全连接层
        self.fc1 = nn.Linear(4 * 4 * 256, 512) # 输入4*4*256， 输出512
        self.fc2 = nn.Linear(512, 256) # 输入512， 输出256
        self.fc3 = nn.Linear(256, 10) # 输入256， 输出10（分10类）
        self.dropout = nn.Dropout(0.5)


    #前向计算
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # print("x_shape:", x.size())
        x = x.view(x.size()[0], -1)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# 实例化模型
model = CNN()
print(model)


CNN(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


## 4.训练卷积神经网络模型

In [ ]:
import torch.optim as optim
import time

# 设置模型的训练次数
n_epochs = 30
# 定义损失函数->交叉熵损失函数
criterion = nn.CrossEntropyLoss()
# 定义优化方法->随机梯度下降
optimizer = optim.SGD(model.parameters(), lr=0.01)

train_losses = []
train_acces = []
eval_losses = []
eval_acces = []

print("start training! \nplease wait for minutes·····")
for epoch in range(n_epochs):
    train_loss = 0.0
    train_acc = 0.0

    # 利用time获取epoch每运行一次所花费的时间
    start_time = time.time()

    ##### 训练集的模型 #####
    model.train()
    for data, target in train_loader:
        # 梯度置0
        optimizer.zero_grad()
        # 正向传播+反向传播+优化
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss


    # 计算平均值
    train_losses.append(train_loss / len(train_loader))

    # 验证集的模型 #
    eval_loss = 0.0
    eval_acc = 0.0

    model.eval()
    for data, target in valid_loader:
        output = model(data)
        loss = criterion(output, target)
        eval_loss += loss

    # 计算平均值
    eval_acces.append(train_acc / len(test_loader))

    # 输出显示训练集与验证集的损失函数
    print('epoch:{}, Train Loss:{:.5f}, '
          'Test Loss:{:.5f}'
          .format(epoch, train_loss / len(train_loader),
                  eval_loss / len(test_loader),
                  ))
    stop_time = time.time()
    print("time is:{:.4f}s".format(stop_time - start_time))


![](https://raw.githubusercontent.com/lilingxuan-dog/7009-neutral-network/main/project2/%E5%9B%BE%E7%89%873.png)

![](https://raw.githubusercontent.com/lilingxuan-dog/7009-neutral-network/main/project2/%E5%9B%BE%E7%89%874.png)

![](https://raw.githubusercontent.com/lilingxuan-dog/7009-neutral-network/main/project2/%E5%9B%BE%E7%89%875.png)
由于测试时间较为慢长，基本每个epoch计算时间保持在2min左右，为节约时间将实验结果直接附上

## 5.类别模型准确率测试
在测试数据上测试训练模型，该训练结果的准确性大约在81%

In [ ]:
# 测试模型
test_loss = 0.0
# 创建一个10维列表，存储后面每个类累积的值
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()
for data, target in test_loader:
    output = model(data)
    loss = criterion(output, target)
    test_loss += loss
    _, pred = torch.max(output, 1)
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy())
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

test_loss = test_loss/len(test_loader)
print('Test Loss:{:.5f}\n'.format(test_loss))
print("End train.")

for i in range(10):
    if class_total[i] > 0:
        print('Test Acc of %5s: %2d%% (%2d/%2d)' % (
            img_class[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Acc of %5s: N/A (no training examples)' % (img_class[i]))

print('\nTest Acc (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))


![](https://raw.githubusercontent.com/lilingxuan-dog/7009-neutral-network/main/project2/%E5%9B%BE%E7%89%876.png)